## Practical: week 6



#### Blocks World Planner: Goal Stack Planning

This notebook implements a simplified planner for the Blocks World domain using goal stack planning. You'll explore symbolic reasoning, STRIPS-style actions, and planning logic. Some parts are left incomplete for you to implement and experiment wi themt

Goal Stack Planning (also called Means-End Analysis) is a non-linear planning algorithm that uses backward chaining to find a plan to get to the goal state from the initial state.
It uses a stack to manage goals and subgoals. It starts from the goal by pushing it onto the stack and works backwards to find actions whose effects achieve it.

The main approach of Goal Stack Planning: 

<ol>
    <li>
        Initialize the stack with the goal state.
    </li>


<li> Pop the top goal from the stack.
</li>
<li> If it’s a compound goal (e.g., conjunction), push each subgoal.
</li>
<li> If it’s a predicate, check if it’s satisfied in the current state:
<ul>
    <li> If yes, do nothing.
<li> If no, find an action that achieves it and push the preconditions of the action and the action itself onto the stack:
</ul>
</li>
<li> Apply actions when their preconditions are satisfied.
</li>
<li> Repeat until the stack is empty.
    </li>
</ol>

For example, for the Blocks World, if the initial state is 
on(A, B), on(B, table), clear(A), clear(C), on(C, table), handempty 
and the goal is 
on(CA,)h.


### Initial State and Goal

For example, for the Blocks World, suppose that the initial state is described by the following state predicates: 
<ul>
    on(A, B), on(B, table), clear(A), clear(C), on(C, table), handempty 
</ul>
and the goal state is descibed by the state predicate we would like to be true: on(C, A).

 You need to complete the code when you **pass**.

In [54]:
# Initial state: initial_state = {'on(A, B)',...}
initial_state = {
    'on(A,B)',
    'ontable(B)',
    'ontable(C)',
    'clear(A)',
    'clear(C)',
    'handempty'
}

# Goal state: goal_state = [...] 
goal_state = [
    'on(C, A)'
]
        

#### STRIPS-style Action Definitions

The STRIPS representation specifies when an action can occur and what the effects of the action are. STRIPS, which stands for “STanford Research Institute Problem Solver,” was the planner used in Shakey, one of the first robots built using AI technology.


The STRIPS representation for an action consi of
sts<ul>
    <li> the precondition, a set of assignments of values to features that must hold for the action to occur, and</li>

   <li> the effect, a set of assignments of values to primitive features that specifies the features that change, and the values they change to, as the result of the action.</li>

<>

on.
The precondition of an action is a proposition – the conjunction of the elements of the set – that must be true before the action is able to be carried out. In terms of constraints, the robot is constrained so it can only choose an action for which the preconditio

For the Blocks World, we have the following STRIPS-representations of four actions, pickup(x), putdown(x), unstack(x, y) and stack(x, y). The preconditions and effects of some of the actions are defined belowTo simplify planning, we  we assume here that the arm can only picka block if it is already on the table. This means that in the initial state given above, the arm would need to take the unstack(C,A) action and put down (C) on table before it can do the pickup(C) action. Without this assumption, we would need to consider two different sets of preconditions for pickup(A) as A could be on table or it could be on another bloclds.

# STRIPS-style actions

Complete the descriptions of the two missing actions of putdown and unstack.

In [55]:
# STRIPS-style actions
actions = {
    'pickup': {
        'pre': ['clear(x)', 'ontable(x)', 'handempty'],
        'add': ['holding(x)'],
        'del': ['ontable(x)', 'clear(x)', 'handempty']
    },
    'putdown': {
        'pre': ['holding()'],
        'add': ['ontable(x)', 'clear(x)','handempty'],
        'del': ['holding(x)']
    },
    'unstack': {
        'pre': ['on(x, y)', 'clear(x)', 'handempty'],
        'add': ['holding(x)', 'clear(y)'],
        'del': ['on(x, y)', 'clear(x)', 'handempty']
    },
    'stake':{
        'pre':['holding(x)', 'clear'],
        'add':['on(x,y)','clear', 'handempty'],
        'del':['holding(x)', 'clear(y)']
    }
}

### Symbolic Unification

The symbolic unification function takes a template like on(x, y) and replaces the variables x and y with actual arguments, say C and A, producing on(C, A). 

This allows abstract action schemas to be instantiated with concrete block names. 

In [56]:
# Symbolic unification
def unify(template, args):
    result = template
    if 'x' in template:
        result = result.replace('x', args[0])
    if 'y' in template and len(args) > 1:
        result = result.replace('y', args[1])
    return result

### Action Application

Action application checks whether an action like pickup(C) can be performed in the current state. It does this by:
<ol>
    <li>Unifying the action’s preconditions (e.g., clear(x), ontable(x), handempty) with the actual arguments (such as C), producing instantiated conditions like clear(C).
</li>
    <li> Verifying that all these conditions are present in the current state so that the action can be taken.</li>
    <li>If they are, it updates the state by removing the delete effects and adding the add effects — simulating the result of executing the action on the given state.</li>
    <li>It also returns a string like pickup(C) to record the action step in the plan.</li>
</ol>


In [57]:
# Apply action to state
def apply_action(action_name, args, state):
    action = actions[action_name]
    pre = [unify(p, args) for p in action['pre']]
    add = [unify(p, args) for p in action['add']]
    delete = [unify(p, args) for p in action['del']]

    if all(p in state for p in pre):
        state = (state - set(delete)) | set(add)
        return state, f"{action_name}({', '.join(args)})"
    else:
        print(f"Skipped invalid action: {action_name}({', '.join(args)}) — preconditions not met.")
        return None, None

### Goal Stack Planner with Backtracking

The goal stack planner a depth-first planner that builds a plan by resolving goals from the top down. It is a recursive, backward-chaining procedure that breaks down high-level goals into subgoals and executable actions. It works by:

<ol>
  <li>
     Popping a goal from the stack and checking if it’s already satisfied in the current state. 
  </li> 
 <li>
If not, it decomposes the goal into either:
actions like pickup(C) or stack(A, B) that can be executed if their preconditions are met;
or predicate subgoals like clear(A) or holding(C) that must be satisfied first before the actionas can be taken.
 <li>
These subgoals are pushed onto the stack, and the planner recursively works through them.
 <li>
When an action is executed, it updates the state and records the action in the plan.
 <li>
If a goal cannot be satisfied, it backtracks to a previous state and tries a different path. 
</ol>


In [58]:
# Goal stack planner
def goal_stack_planner(initial_state, goal_state):
    from copy import deepcopy

    goal_stack = goal_state.copy()
    history = []
    current_state = deepcopy(initial_state)
    plan = []

    while goal_stack:
        goal = goal_stack.pop()

        print(f"Processing goal: {goal}")
        print(f"Current state: {current_state}")
        print(f"Goal stack: {goal_stack}")
        print(f"Plan so far: {plan}")
        print("------")

        if goal in current_state:
            continue
        # Action execution
        if '(' in goal and goal.endswith(')'):
            action_name = goal[:goal.index('(')]
            args = goal[goal.index('(')+1:-1].split(', ')
            if action_name in actions:
                new_state, action_str = apply_action(action_name, args, current_state)
                if new_state:
                    current_state = new_state
                    plan.append(action_str)
                    continue
                elif history:
                    current_state, goal_stack, plan = history.pop()
                    continue
                else:
                    print("No valid plan found.")
                    return []

        # Save state for backtracking
        history.append((deepcopy(current_state), deepcopy(goal_stack), deepcopy(plan)))

        # Predicate goals
        if goal.startswith('on('):
            x, y = goal[3:-1].split(', ')
            goal_stack.extend([f'stack({x}, {y})', f'holding({x})', f'clear({y})'])

        elif goal.startswith('holding('):
            x = goal[8:-1]
            if f'ontable({x})' in current_state:
                goal_stack.extend([f'pickup({x})', f'clear({x})', f'ontable({x})', 'handempty'])
            else:
                for fact in current_state:
                    if fact.startswith(f'on({x}, '):
                        y = fact[3:-1].split(', ')[1]
                        goal_stack.extend([f'unstack({x}, {y})', f'on({x}, {y})', f'clear({x})', 'handempty'])
                        break

        elif goal.startswith('clear('):
            x = goal[6:-1]
            for fact in current_state:
                if fact.startswith('on(') and fact.endswith(f', {x})'):
                    y = fact[3:-1].split(', ')[0]
                    goal_stack.extend([f'unstack({y}, {x})'])
                    break

        elif goal.startswith('ontable('):
            x = goal[8:-1]
            if f'holding({x})' in current_state:
                goal_stack.extend([f'putdown({x})'])

        elif goal == 'handempty':
            for fact in current_state:
                if fact.startswith('holding('):
                    x = fact[8:-1]
                    goal_stack.extend([f'putdown({x})'])
                    break

    return plan

### Run the Planner

In [59]:
plan = goal_stack_planner(initial_state, goal_state)

print("Generated Plan:")
for step in plan:
    print(step)


Processing goal: on(C, A)
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'clear(A)', 'handempty', 'clear(C)'}
Goal stack: []
Plan so far: []
------
Processing goal: clear(A)
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'clear(A)', 'handempty', 'clear(C)'}
Goal stack: ['stack(C, A)', 'holding(C)']
Plan so far: []
------
Processing goal: holding(C)
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'clear(A)', 'handempty', 'clear(C)'}
Goal stack: ['stack(C, A)']
Plan so far: []
------
Processing goal: handempty
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'clear(A)', 'handempty', 'clear(C)'}
Goal stack: ['stack(C, A)', 'pickup(C)', 'clear(C)', 'ontable(C)']
Plan so far: []
------
Processing goal: ontable(C)
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'clear(A)', 'handempty', 'clear(C)'}
Goal stack: ['stack(C, A)', 'pickup(C)', 'clear(C)']
Plan so far: []
------
Processing goal: clear(C)
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'cle

In [60]:
# Compound goal ordering 1
print("Attempting plan with goal ordering: ['on(A, B)', 'on(C, A)']")

plan2 = goal_stack_planner(initial_state, ['on(A,B)','on(C,A)'])

print("Plan with ordering ['on(A, B)', 'on(C, A)']:")

for step in plan2:
    print(step)

Attempting plan with goal ordering: ['on(A, B)', 'on(C, A)']
Processing goal: on(C,A)
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'clear(A)', 'handempty', 'clear(C)'}
Goal stack: ['on(A,B)']
Plan so far: []
------


ValueError: not enough values to unpack (expected 2, got 1)

In [49]:
# Compound goal ordering 2
print("Attempting plan with goal ordering: ['on(C, A)', 'on(A, B)']")

plan1 = goal_stack_planner(initial_state, ['on(C,A)','on(A,B)'])

print("Plan with ordering ['on(C, A)', 'on(A, B)']:")

for step in plan1:
    print(step)

Attempting plan with goal ordering: ['on(C, A)', 'on(A, B)']
Processing goal: on(A,B)
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'clear(A)', 'handempty', 'clear(C)'}
Goal stack: ['on(C,A)']
Plan so far: []
------
Processing goal: on(C,A)
Current state: {'on(A,B)', 'ontable(B)', 'ontable(C)', 'clear(A)', 'handempty', 'clear(C)'}
Goal stack: []
Plan so far: []
------


ValueError: not enough values to unpack (expected 2, got 1)

### Next Steps You Might Explore

Introduce a goal that requires undoing a satisfied fact, like clear(B) when A is on B, and generate a plan.

Add a compound goal that temporarily conflicts, e.g., on(C, A) and clear(A) and generate a plan. Why does this plan violate clear(A)? How could we fix it?